# Postgresql

In [219]:
import psycopg2
import uuid
import time
def openDB():
    try:
        conn = psycopg2.connect(database='postgres', user = "postgres", password = "mysecretpassword", host = "127.0.0.1", port = "5432")
        print ("Opened database successfully")
        return conn
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print("Error! Open database.")

    return None

In [220]:
def closeDB(conn):
    try:
        conn.close()
        print ("Closed database successfully")
    except:
        print("Error! Close database.")

In [221]:
def dropTable(conn):
    cur = conn.cursor()
    try:
        cur.execute("DROP TABLE IF EXISTS news")
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        print("Drop table, successful!")
        return True
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print("Error! cannot drop the table.")
        return False

conn = openDB()
dropTable(conn)
closeDB(conn)

Opened database successfully
Drop table, successful!
Closed database successfully


In [222]:
conn = openDB()

cur = conn.cursor()
if conn is not None:
    sql_create_entry_table = """ CREATE TABLE IF NOT EXISTS news (
        _id text PRIMARY KEY,
        title text NOT NULL,
        datatext text NOT NULL,
        source text NOT NULL,
        created_at timestamp  NOT NULL  DEFAULT current_timestamp,
        not_deleted  boolean DEFAULT true,
        deleted_at   timestamp
        ); """
    cur.execute(sql_create_entry_table)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
    print("Table successful created.")
else:
    print("Error! cannot create table .")
closeDB(conn)

Opened database successfully
Table successful created.
Closed database successfully


In [223]:
def insertData(conn, ids, title, desc, source):
    cur = conn.cursor()
    sql2 = """INSERT INTO news  (
                                _id,
                                title, 
                                datatext, 
                                source
                                ) VALUES (%s, %s, %s, %s)  ON CONFLICT DO NOTHING"""
    task = (ids, title, desc, source)
    vendor_id = None
    try:
        cur.execute(sql2, task)
        # get the generated id back
        #vendor_id = cur.fetchone()[0]
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        print("Success insert data.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print("Error! insert data.")
    return vendor_id

conn = openDB()
ids = str(uuid.uuid4())+str(time.time())
ids
insertData(conn, ids, "title", "desc", "asal")
closeDB(conn)

Opened database successfully
Success insert data.
Closed database successfully


In [224]:
def insertListData(conn, listdata):
    """ insert multiple news into the news table  """
    sql = "INSERT INTO news(_id, title, datatext, source) VALUES(%s, %s, %s, %s)"
    try:
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.executemany(sql,listdata)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        print("Successful insert list data.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print("Error! insert list data.")


conn = openDB()
insertListData(conn, [
        (str(uuid.uuid4())+str(time.time()), "title1", "desc1", "asal1"),
        (str(uuid.uuid4())+str(time.time()), "title2", "desc2", "asal2"),
        (str(uuid.uuid4())+str(time.time()), "title3", "desc3", "asal3"),
        (str(uuid.uuid4())+str(time.time()), "title4", "desc4", "asal4"),
        (str(uuid.uuid4())+str(time.time()), "title5", "desc5", "asal5")
    ])

closeDB(conn)

Opened database successfully
Successful insert list data.
Closed database successfully


In [225]:
def getData(conn):
    cur = conn.cursor()
    sql_entry_select_by_data = "SELECT * FROM news"
    data = cur.execute(sql_entry_select_by_data)

    hasil = []
    for row in cur.fetchall():
        hasil.append(row)
    cur.close()
    return hasil
conn = openDB()
data = getData(conn)
closeDB(conn)
data

Opened database successfully
Closed database successfully


[('7458371b-aa27-4a0f-8586-83e3b69edef91532509588.99',
  'title',
  'desc',
  'asal',
  datetime.datetime(2018, 7, 25, 9, 6, 28, 991151),
  True,
  None),
 ('2931cf00-a433-447b-8b63-6223eb45e8db1532509589.01',
  'title1',
  'desc1',
  'asal1',
  datetime.datetime(2018, 7, 25, 9, 6, 29, 14605),
  True,
  None),
 ('8cd23582-a48e-434c-a64a-0b98902298fa1532509589.01',
  'title2',
  'desc2',
  'asal2',
  datetime.datetime(2018, 7, 25, 9, 6, 29, 14605),
  True,
  None),
 ('735d7ed8-94bc-497f-97e6-9914d7b505021532509589.01',
  'title3',
  'desc3',
  'asal3',
  datetime.datetime(2018, 7, 25, 9, 6, 29, 14605),
  True,
  None),
 ('ba93af1a-7911-4139-828e-c5778d57268b1532509589.01',
  'title4',
  'desc4',
  'asal4',
  datetime.datetime(2018, 7, 25, 9, 6, 29, 14605),
  True,
  None),
 ('d40304d4-89d2-4494-aff8-d8a5727df4251532509589.01',
  'title5',
  'desc5',
  'asal5',
  datetime.datetime(2018, 7, 25, 9, 6, 29, 14605),
  True,
  None)]

In [226]:
def getDataById(conn, ids):
    cur = conn.cursor()
    sql_entry_select_by_data = "SELECT * FROM news WHERE _id ILIKE %s"
    data = cur.execute(sql_entry_select_by_data,(ids,))

    hasil = []
    for row in cur.fetchall():
        hasil.append(row)
    cur.close()
    return hasil
conn = openDB()
data = getDataById(conn, 'd198bd14-81ce-4d84-bc8c-c6ee00e895191532508482.41')
closeDB(conn)
data

Opened database successfully
Closed database successfully


[]

In [227]:
def updateData(conn, ids, title, desc, sources):
    cur = conn.cursor()
    """ update news name based on the news id """
    sql = """ UPDATE news SET title = %s WHERE _id ILIKE %s"""
    updated_rows = 0
    try:
        # execute the UPDATE  statement
        cur.execute(sql, (title, ids))
        # get the number of updated rows
        updated_rows = cur.rowcount
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        
        print("Successful update data.")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        print("Error! updated data.")
 
    return ids, updated_rows

conn = openDB()

data = getData(conn)
print(data[0][0],data[0][1],data[0][2],data[0][3])

ids, numDataUpdated = updateData(conn,data[0][0],"data[0]assdda[1]aku",data[0][2],data[0][3])
data = getDataById(conn, ids)
print(data[0][0],data[0][1],data[0][2],data[0][3])

closeDB(conn)

Opened database successfully
('7458371b-aa27-4a0f-8586-83e3b69edef91532509588.99', 'title', 'desc', 'asal')
Successful update data.
('7458371b-aa27-4a0f-8586-83e3b69edef91532509588.99', 'data[0]assdda[1]aku', 'desc', 'asal')
Closed database successfully


In [231]:
def deleteData(conn, part_id):
    """ delete news by news id """
    sql = "DELETE FROM news WHERE _id = %s"
    rows_deleted = 0
    try:
        # create a new cursor
        cur = conn.cursor()
        # execute the UPDATE  statement
        cur.execute(sql, (part_id,))
        # get the number of updated rows
        rows_deleted = cur.rowcount
        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
 
    return part_id, rows_deleted

conn = openDB()

ids, numDataUpdated = deleteData(conn,"4c668f59-b445-4c13-b78f-91c6ec5665a41532508482.41")
print(ids, numDataUpdated)
closeDB(conn)

Opened database successfully
('4c668f59-b445-4c13-b78f-91c6ec5665a41532508482.41', 0)
Closed database successfully


# MongoDb

In [297]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime

client = MongoClient("mongodb://127.0.0.1:27017/")
print(client)

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)


In [290]:
collection = client['news']['article']

def insertDataMongo(collection, title, desc, source):
    post = {
            "title": title,
            "desc": desc,
            "source": source,
            "validated": False,
            "timestamp": datetime.datetime.utcnow(),
            "updated": datetime.datetime.utcnow()
        }
    
    try:
        key = {'title': title, 'desc': desc, "source": source}
        onedata = collection.find_one(key)
        if onedata:
            #collection.replace_one(key, post, upsert=True) #Use replace_one, update_one or update_many instead.
            print("data not inserted, data exist >>> ", post)
        else:
            post_id = collection.insert_one(post).inserted_id
            print("data inserted >>> ", post_id, post)
    except:
        print("data error >> ", x)  # pass
        
insertDataMongo(collection, "titleasdsd", "desc", "source")

('data inserted >>> ', ObjectId('5b58471853167b241c0d13b3'), {'updated': datetime.datetime(2018, 7, 25, 9, 47, 4, 985856), '_id': ObjectId('5b58471853167b241c0d13b3'), 'title': 'titleasdsd', 'timestamp': datetime.datetime(2018, 7, 25, 9, 47, 4, 985854), 'source': 'source', 'validated': False, 'desc': 'desc'})


In [310]:
def insertListDataMongo(collection, listdata):
    try:
        x = collection.insert_many(listdata)

        #print list of the _id values of the inserted documents:
        print(x.inserted_ids)
    except:
        print("data error list inserted")
        
mylist = [
  { "title": "title1", "desc": "desc1", "source": "source1", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title2", "desc": "desc2", "source": "source2", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title3", "desc": "desc3", "source": "source3", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title4", "desc": "desc4", "source": "source4", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title5", "desc": "desc5", "source": "source5", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title6", "desc": "desc6", "source": "source6", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title7", "desc": "desc7", "source": "source7", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title8", "desc": "desc8", "source": "source8", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title9", "desc": "desc9", "source": "source9", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title10", "desc": "desc10", "source": "source10", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title11", "desc": "desc11", "source": "source11", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title12", "desc": "desc12", "source": "source12", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()},
  { "title": "title13", "desc": "desc13", "source": "source13", "validated": False, "timestamp": datetime.datetime.utcnow(), "updated": datetime.datetime.utcnow()}
]
insertListDataMongo(collection, mylist)

[ObjectId('5b584c4353167b241c0d13b5'), ObjectId('5b584c4353167b241c0d13b6'), ObjectId('5b584c4353167b241c0d13b7'), ObjectId('5b584c4353167b241c0d13b8'), ObjectId('5b584c4353167b241c0d13b9'), ObjectId('5b584c4353167b241c0d13ba'), ObjectId('5b584c4353167b241c0d13bb'), ObjectId('5b584c4353167b241c0d13bc'), ObjectId('5b584c4353167b241c0d13bd'), ObjectId('5b584c4353167b241c0d13be'), ObjectId('5b584c4353167b241c0d13bf'), ObjectId('5b584c4353167b241c0d13c0'), ObjectId('5b584c4353167b241c0d13c1')]


In [288]:
def updateDataMongo(collection, title, desc, source):
    post = {
            "title": title,
            "desc": desc,
            "source": source,
            "validated": False,
            "timestamp": datetime.datetime.utcnow(),
            "updated": datetime.datetime.utcnow()
        }
    
    try:
        key = {'title': title, 'desc': desc, "source": source}
        onedata = collection.find_one(key)
        if onedata:
            collection.replace_one(key, post, upsert=True) #Use replace_one, update_one or update_many instead.
            print("data updated >>> ", post)
        else:
            print("data not updated, data not exist >>> ", post)
            #post_id = collection.insert_one(post).inserted_id
    except:
        print("data error >> ", x)  # pass
        
updateDataMongo(collection, "titldasdasde", "desc", "source")

('data not updated, data not exist >>> ', {'updated': datetime.datetime(2018, 7, 25, 9, 41, 54, 170118), 'title': 'titldasdasde', 'timestamp': datetime.datetime(2018, 7, 25, 9, 41, 54, 170117), 'source': 'source', 'validated': False, 'desc': 'desc'})


In [308]:
def getData(collection, ids):
    try:
        key = {'_id': ObjectId(ids)}
        gate = collection.find(key)
        onedata = collection.find_one(key)
        return onedata
    except:
        return {}

getData(collection, "5b584312501d54c3d1abc43f")

{u'_id': ObjectId('5b584312501d54c3d1abc43f'),
 u'desc': False,
 u'source': u'data',
 u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 928000),
 u'title': u'aku',
 u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 928000),
 u'validated': False}

In [317]:
def getDataList(collection, source):
    try:
        key = {'source': source}
        gate = collection.find(key)
        onedata = collection.find(key)
        return onedata
    except:
        return {}

data = getDataList(collection, "data")
for x in data:
    print x

{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 928000), u'validated': False, u'title': u'aku', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 928000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc43f'), u'desc': False}
{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 960000), u'validated': False, u'title': u'mau', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 960000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc442'), u'desc': False}
{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 962000), u'validated': False, u'title': u'makan', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 962000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc445'), u'desc': False}
{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 964000), u'validated': False, u'title': u'nasi', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 964000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc448'), u'desc': Fals

In [316]:
def getDataAllList(collection):
    try:
        onedata = collection.find()
        return onedata
    except:
        return {}

data = getDataAllList(collection)
for x in data:
    print x

{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 928000), u'validated': False, u'title': u'aku', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 928000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc43f'), u'desc': False}
{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 960000), u'validated': False, u'title': u'mau', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 960000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc442'), u'desc': False}
{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 962000), u'validated': False, u'title': u'makan', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 962000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc445'), u'desc': False}
{u'updated': datetime.datetime(2018, 7, 25, 9, 29, 54, 964000), u'validated': False, u'title': u'nasi', u'timestamp': datetime.datetime(2018, 7, 25, 9, 29, 54, 964000), u'source': u'data', u'_id': ObjectId('5b584312501d54c3d1abc448'), u'desc': Fals

In [318]:
def deleteData(collection, ids):
    myquery = { "_id": ObjectId(ids) }

    x = collection.delete_one(myquery)
    print(x.deleted_count, " documents deleted.")


In [319]:
def deleteManyData(collection, key, value):
    myquery = { key: value }

    x = collection.delete_one(myquery)
    print(x.deleted_count, " documents deleted.")

In [322]:
def deleteAllData(collection):
    x = collection.delete_many({})

    print(x.deleted_count, " documents deleted.")
deleteAllData(collection)

(25, ' documents deleted.')


# Sqlite

In [51]:
import sqlite3
import uuid
import time
DB_NAME = "newsCorpus.db"
DB_VERSION = 1

def openSqliteDB():
    try:
        conn = sqlite3.connect(DB_NAME)
#         conn = sqlite3.connect(':memory:')
        print ("Opened database successfully")
        return conn
    except sqlite3.Error as e:
        print(e)
        print("Error! Open database.")
        return None

In [52]:
def closeSqliteDB(conn):
    try:
        conn.close()
        print ("Closed database successfully")
    except:
        print("Error! Close database.")

In [53]:
def dropSqliteTable(conn):
    try:
        cur = conn.cursor()
        cur.execute('''DROP TABLE IF EXISTS news''')
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        print("Drop table, successful!")
        return True
    except (Exception, sqlite3.Error) as error:
        print(error)
        print("Error! cannot drop the table.")
        return False

conn = openSqliteDB()
dropSqliteTable(conn)
closeSqliteDB(conn)

Opened database successfully
Drop table, successful!
Closed database successfully


In [54]:
def createSqliteTable(conn):
    sql = ''' CREATE TABLE IF NOT EXISTS news (
        _id text PRIMARY KEY,
        title text NOT NULL,
        datatext text NOT NULL,
        source text NOT NULL,
        created_at timestamp  NOT NULL  DEFAULT current_timestamp,
        not_deleted  boolean DEFAULT true,
        deleted_at   timestamp
        ); '''
    try:
        cur = conn.cursor()
        cur.execute(sql)
        # commit the changes
        conn.commit()
        # close communication with the PostgreSQL database server
        cur.close()
        print("Table successful created.")
    except (Exception, sqlite3.Error) as error:
        print("Error! cannot create table :"+error)

conn = openSqliteDB()
createSqliteTable(conn)
closeSqliteDB(conn)

Opened database successfully
Table successful created.
Closed database successfully


In [60]:
def insertSqliteData(conn, ids, title, desc, source ):
    cur = conn.cursor()
    sql2 = """INSERT INTO {tn}  (
                                {cid},
                                {ct}, 
                                {cdt}, 
                                {cs}
                                ) VALUES (?, ?, ?, ?)"""
    task = (ids, title, desc, source)
    vendor_id = None
    try:
        hasil = cur.execute(sql2.format(tn="news", cid="_id", ct="title", cdt="datatext", cs="source"), task)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        print("Success insert data.")
    except (Exception, sqlite3.Error) as error:
        print(error)
        print("Error! insert data.")
    return vendor_id

conn = openSqliteDB()
ids = str(uuid.uuid4())+str(time.time())
ids
insertSqliteData(conn, ids, "title", "desc", "asal")
closeSqliteDB(conn)

Opened database successfully
Success insert data.
Closed database successfully


In [61]:
def insertSqliteListData(conn, listdata):
    """ insert multiple news into the news table  """
    sql = "INSERT INTO news(_id, title, datatext, source) VALUES(?, ?, ?, ?)"
    try:
        # create a new cursor
        cur = conn.cursor()
        # execute the INSERT statement
        cur.executemany(sql,listdata)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        print("Successful insert list data.")
    except (Exception, sqlite3.Error) as error:
        print(error)
        print("Error! insert list data.")


conn = openSqliteDB()
insertSqliteListData(conn, [
        (str(uuid.uuid4())+str(time.time()), "title1", "desc1", "asal1"),
        (str(uuid.uuid4())+str(time.time()), "title2", "desc2", "asal2"),
        (str(uuid.uuid4())+str(time.time()), "title3", "desc3", "asal3"),
        (str(uuid.uuid4())+str(time.time()), "title4", "desc4", "asal4"),
        (str(uuid.uuid4())+str(time.time()), "title5", "desc5", "asal5")
    ])

closeSqliteDB(conn)

Opened database successfully
Successful insert list data.
Closed database successfully


In [63]:
def getSqliteData(conn):
    cur = conn.cursor()
    sql_entry_select_by_data = "SELECT * FROM news"
    data = cur.execute(sql_entry_select_by_data)

    hasil = []
    for row in cur.fetchall():
        hasil.append(row)
    cur.close()
    return hasil

conn = openSqliteDB()
data = getSqliteData(conn)
closeSqliteDB(conn)
data

Opened database successfully
Closed database successfully


[(u'ee825cfc-ce06-4228-8cf2-1484727087611532569676.14',
  u'title',
  u'desc',
  u'asal',
  u'2018-07-26 01:47:56',
  u'true',
  None),
 (u'766e996f-656e-4ae3-92bd-4fabd24841901532569708.5',
  u'title',
  u'desc',
  u'asal',
  u'2018-07-26 01:48:28',
  u'true',
  None),
 (u'0ba65020-a9d8-4d05-9621-463828928b6e1532569712.59',
  u'title',
  u'desc',
  u'asal',
  u'2018-07-26 01:48:32',
  u'true',
  None),
 (u'd4664e71-5d51-4830-8466-d5701ea259af1532569731.65',
  u'title',
  u'desc',
  u'asal',
  u'2018-07-26 01:48:51',
  u'true',
  None),
 (u'c863ce25-92be-4e52-b1f5-76eb9f9012b81532569736.48',
  u'title',
  u'desc',
  u'asal',
  u'2018-07-26 01:48:56',
  u'true',
  None),
 (u'f028a42b-937d-46e4-897e-fa79e7c885411532569804.81',
  u'title',
  u'desc',
  u'asal',
  u'2018-07-26 01:50:04',
  u'true',
  None),
 (u'11953d08-85c9-4d2b-ad27-a28b6ad49a191532570041.37',
  u'title1',
  u'desc1',
  u'asal1',
  u'2018-07-26 01:54:01',
  u'true',
  None),
 (u'63adf535-e61f-4aa3-87c0-6c5837fc5c31153257

In [67]:
def getSqliteDataById(conn, ids):
    cur = conn.cursor()
    sql_entry_select_by_data = "SELECT * FROM news WHERE _id = ?"
    data = cur.execute(sql_entry_select_by_data,(ids,))

    hasil = []
    for row in cur.fetchall():
        hasil.append(row)
    cur.close()
    return hasil
conn = openSqliteDB()
data = getSqliteDataById(conn, '76d1166a-93d3-4386-ba2b-737831fe0e511532570041.37')
closeSqliteDB(conn)
data

Opened database successfully
Closed database successfully


[(u'76d1166a-93d3-4386-ba2b-737831fe0e511532570041.37',
  u'data[0]assdda[1]aku',
  u'desc5',
  u'asal5',
  u'2018-07-26 01:54:01',
  u'true',
  None)]

In [66]:
def updateSqliteData(conn, ids, title, desc, sources):
    cur = conn.cursor()
    """ update news name based on the news id """
    sql = """ UPDATE news SET title = ?, datatext = ?, source = ? WHERE _id = ?"""
    updated_rows = 0
    try:
        # execute the UPDATE  statement
        cur.execute(sql, (title, desc, sources, ids))
        # get the number of updated rows
        updated_rows = cur.rowcount
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
        
        print("Successful update data.")
    except (Exception, sqlite3.Error) as error:
        print(error)
        print("Error! updated data.")
 
    return ids, updated_rows

conn = openSqliteDB()

data = getSqliteDataById(conn, '76d1166a-93d3-4386-ba2b-737831fe0e511532570041.37')
print(data[0][0],data[0][1],data[0][2],data[0][3])

ids, numDataUpdated = updateSqliteData(conn,data[0][0],"data[0]assdda[1]aku",data[0][2],data[0][3])
data = getSqliteDataById(conn, '76d1166a-93d3-4386-ba2b-737831fe0e511532570041.37')
print(data[0][0],data[0][1],data[0][2],data[0][3])

closeSqliteDB(conn)

Opened database successfully
(u'76d1166a-93d3-4386-ba2b-737831fe0e511532570041.37', u'title5', u'desc5', u'asal5')
Successful update data.
(u'76d1166a-93d3-4386-ba2b-737831fe0e511532570041.37', u'data[0]assdda[1]aku', u'desc5', u'asal5')
Closed database successfully


In [73]:
def deleteData(conn, part_id):
    """ delete news by news id """
    sql = "DELETE FROM news WHERE _id = ?"
    rows_deleted = 0
    try:
        # create a new cursor
        cur = conn.cursor()
        # execute the UPDATE  statement
        cur.execute(sql, (part_id,))
        # get the number of updated rows
        rows_deleted = cur.rowcount
        # Commit the changes to the database
        conn.commit()
        # Close communication with the PostgreSQL database
        cur.close()
        print("Successful deleted "+part_id+" data.")
    except (Exception, sqlite3.Error) as error:
        print(error)
        print("Error! deleted "+part_id+" data.")
 
    return part_id, rows_deleted

conn = openSqliteDB()

ids, numDataUpdated = deleteData(conn,"63adf535-e61f-4aa3-87c0-6c5837fc5c311532570041.37")
print(ids, numDataUpdated)
data = getSqliteDataById(conn, '63adf535-e61f-4aa3-87c0-6c5837fc5c311532570041.37')
print(data)
closeSqliteDB(conn)

Opened database successfully
Successful deleted 63adf535-e61f-4aa3-87c0-6c5837fc5c311532570041.37 data.
('63adf535-e61f-4aa3-87c0-6c5837fc5c311532570041.37', 0)
[]
Closed database successfully


# Sql Alchemy (for SQL Database)

In [52]:
# http://docs.sqlalchemy.org/en/latest/core/engines.html
from sqlalchemy import exc, and_, create_engine, MetaData, event, Column, Table, Integer, String, ForeignKey, DateTime, Text
from sqlalchemy.sql import func
from sqlalchemy.engine import Engine
import logging, uuid, time

SQLITE_DB_NAME = 'sqlite:///corpus.db'

PG_USER = "postgres"
PG_PASS = "mysecretpassword"
PG_HOST = "127.0.0.1"
PG_PORT = "5432"
PG_DB = "postgres"

# https://www.compose.com/articles/using-postgresql-through-sqlalchemy/
# https://suhas.org/sqlalchemy-tutorial/
POSTGRESQL_DB_NAME = 'postgresql://{}:{}@{}:{}/{}'
POSTGRESQL_DB_NAME = POSTGRESQL_DB_NAME.format(PG_USER, PG_PASS, PG_HOST, PG_PORT, PG_DB)

# https://stackoverflow.com/questions/9447629/mongokit-vs-mongoengine-vs-flask-mongoalchemy-for-flask
# https://pythonhosted.org/Flask-MongoAlchemy/



DB_USED = POSTGRESQL_DB_NAME #SQLITE_DB_NAME

### Sql Alchemy

In [39]:
# @event.listens_for(Engine, "connect")
# def set_sqlite_pragma(dbapi_connection, connection_record):
#     cursor = dbapi_connection.cursor()
#     cursor.execute("PRAGMA foreign_keys=ON")
#     cursor.close()

In [40]:
#https://www.pythoncentral.io/introductory-tutorial-python-sqlalchemy/

from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship


Base = declarative_base()
 
class News(Base):
    __tablename__ = 'news2'
    # Here we define columns for the table person
    # Notice that each column is also a normal Python instance attribute.
    _id = Column(String(64), primary_key=True, nullable=False)
    title = Column(Text, nullable=False)
    datatext = Column(Text, nullable=False)
    source = Column(String(64), nullable=False)
    aktif = Column(Integer, nullable=False, default=1)
    created_at = Column(DateTime(timezone=True), default=func.now())
    update_at = Column(DateTime(timezone=True), onupdate=func.now(), default=func.now())
    
engine = create_engine(DB_USED)
Base.metadata.create_all(engine)

In [41]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine(DB_USED)
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
 
DBSession = sessionmaker(bind=engine)
# A DBSession() instance establishes all conversations with the database
# and represents a "staging zone" for all the objects loaded into the
# database session object. Any change made against the objects in the
# session won't be persisted into the database until you call
# session.commit(). If you're not happy about the changes, you can
# revert all of them back to the last commit by calling
# session.rollback()
session = DBSession()

In [42]:
# Insert a Person in the person table
new_news = News(_id=str(uuid.uuid4())+str(time.time()), title="title", datatext="text", source="source")
session.add(new_news)
session.commit()

In [46]:
news = session.query(News).all()
news
for x in news:
    n = x._id #news.title
    print (x._id, x.title, x.datatext, x.source)

(u'4bd32563-36d6-445d-983d-a2a9406d2d491532576627.51', u'title', u'text', u'source')
(u'86716db2-1750-49e0-a626-53e913fdd9b21532576632.03', u'title', u'text', u'source')
(u'5e8be8c7-9872-4a05-a535-414801e26f1d1532576671.19', u'title12', u'text', u'source')


In [47]:
news = session.query(News).filter(News._id == "96fc17b7-3de2-40e1-bf23-2cbea40262a21532572572.03").all()
news
for x in news:
    n = x.title #news.title
    print (n)

In [48]:
session.query(News).filter_by(_id=u"5e8be8c7-9872-4a05-a535-414801e26f1d1532576671.19").update({"title": u"title12"})

1

In [49]:
news = session.query(News).filter_by(_id=u"96fc17b7-3de2-40e1-bf23-2cbea40262a21532572572.03").delete()
news

0

# ElasticSearch

In [98]:
# run elastic server (better run with docker for learning) 
# docker run -d -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" elasticsearch
# Open Chrome extension:  Elastic - "Head" 
# https://chrome.google.com/webstore/detail/elasticsearch-head/ffmkiejjmecolpfloofpjologoblkegm

# https://elasticsearch-py.readthedocs.io/en/master/
# https://towardsdatascience.com/getting-started-with-elasticsearch-in-python-c3598e718380
from elasticsearch import Elasticsearch
server, port, timeout = '10.226.174.30', 9200, 30
try:
    conEs = Elasticsearch([{'host':server,'port':port,'timeout':timeout}])
    if conEs.ping():
        print('Sip! Elastic Server Connect')
    else:
        print('Error, not connect to ElasticSearch. please check your server, ip, and port')
except:
    print('Appications error')

Sip! Elastic Server Connect


In [99]:
# Create index (~table in RDBMS) in Elastic
index_name="twitter"
docType="tweets" # Warning!, "doc_type" removed in next version
nShards = 2
nReplicas = 0
# https://www.elastic.co/guide/en/elasticsearch/reference/current/index-modules-similarity.html
similarity="BM25"

In [74]:
# Create setting index
dSetting={
        "settings":{
            "index":{
                "number_of_shards":nShards,
                "number_of_replicas":nReplicas
            }
        },
        "analysis":{
            "analyzer":{
                "english":{
                    "filter":["lowercase","indonesian_stop","indonesian_stemmer"]
                }
            }
        }
}

ttestsettings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "members": {
                "dynamic": "strict",
                "properties": {
                    "title": {
                        "type": "text"
                    },
                    "submitter": {
                        "type": "text"
                    },
                    "description": {
                        "type": "text"
                    },
                    "calories": {
                        "type": "integer"
                    },
                }
            }
        }
    }
# Yes!, ElasticSearch support Bahasa Indonesia
# https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis-lang-analyzer.html
print('Done!')

Done!
Index created


In [75]:
# Oleh karena itu, "best practice"-nya tetap kita membutuhkan "mapping setting"
# Mapping setting bisa dilakukan ke sebagian field, lalu sisanya automatic by default
mapping={docType:{
    "properties":{
        "full_text":{
            "type":"text",
            "store":True,
            "term_vector":"yes",
            "similarity":"BM25"
        },
        "created_at":{
            "type": "date", 
            "store":True, 
            "format": "EEE MMM dd HH:mm:ss Z YYYY"
        },
        "screen_name":{
            "type":"text"
        }
    }}}
# Perhatikan saya set "term_vector" di definisi mapping indexnya, 
# kita mulai masuk ke inti penggunaan elastic di Text Mining dengan "term_vector".
# term_vector bukan default, jadi harus secara eksplisit dinyatakan dalam mapping index-nya

In [78]:
try:
    # create new index in Elastic
  # # conEs.indices.delete(index=index_name,ignore=[400, 404]) # Deleted index if exist
    # Ignore 400 means to ignore "Index Already Exist" error.
    conEs.indices.create(index=index_name,body=dSetting,ignore=400) # create  the index
    conEs.indices.put_mapping(index=idx,doc_type=docType,body=mapping)
    # refresh elastic head in chrome extentions
    print('Index created')
except Exception as ex:
    print(str(ex))
    print('Error! Index not created')

Index created


In [ ]:
Slide Type

# Bentuk tweet = Json. Contoh tweet pertama:
Tweets[0]
{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Jul 26 04:56:14 +0000 2018',
 'display_text_range': [11, 64],
 'entities': {'hashtags': [],
  'symbols': [],
  'urls': [],
  'user_mentions': [{'id': 2264244858,
    'id_str': '2264244858',
    'indices': [0, 10],
    'name': 'ajiq',
    'screen_name': 'aziqz9513'}]},
 'favorite_count': 0,
 'favorited': False,
 'full_text': '@aziqz9513 Tu ah aku jadi model tuk surgery tuk jadi hensem hmmm',
 'geo': None,
 'id': 1022344821162237957,
 'id_str': '1022344821162237957',
 'in_reply_to_screen_name': 'aziqz9513',
 'in_reply_to_status_id': 1022343422676357120,
 'in_reply_to_status_id_str': '1022343422676357120',
 'in_reply_to_user_id': 2264244858,
 'in_reply_to_user_id_str': '2264244858',
 'is_quote_status': False,
 'lang': 'in',
 'metadata': {'iso_language_code': 'in', 'result_type': 'recent'},
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Mon Apr 01 17:07:34 +0000 2013',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'Hari hari makan minum',
  'entities': {'description': {'urls': []}},
  'favourites_count': 4274,
  'follow_request_sent': False,
  'followers_count': 263,
  'following': False,
  'friends_count': 307,
  'geo_enabled': False,
  'has_extended_profile': True,
  'id': 1320763153,
  'id_str': '1320763153',
  'is_translation_enabled': False,
  'is_translator': False,
  'lang': 'en',
  'listed_count': 1,
  'location': '',
  'name': 'Sudin aklakam',
  'notifications': False,
  'profile_background_color': 'C0DEED',
  'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
  'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
  'profile_background_tile': False,
  'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1320763153/1508955695',
  'profile_image_url': 'http://pbs.twimg.com/profile_images/1008695387975180288/zHoTPbEE_normal.jpg',
  'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1008695387975180288/zHoTPbEE_normal.jpg',
  'profile_link_color': '9C27B0',
  'profile_sidebar_border_color': '000000',
  'profile_sidebar_fill_color': 'DDEEF6',
  'profile_text_color': '333333',
  'profile_use_background_image': True,
  'protected': False,
  'screen_name': 's_udin73',
  'statuses_count': 8022,
  'time_zone': None,
  'translator_type': 'none',
  'url': None,
  'utc_offset': None,
  'verified': False}}

In [79]:
# # lalu kita index/simpan ulang datanya, misal sekarang hanya menyimpan full_text, created_at, dan screen_name saja
# for tweet in tqdm(Tweets):
#     doc = {'created_at':tweet['created_at'], 'screen_name':tweet['user']['screen_name'], 'full_text':tweet['full_text']}
#     conEs.index(index=idx,doc_type=docType,body=doc)
# # Setelah selesai, coba cek hasilnya di Plugin Head: Browser dan index metadata

In [130]:
# query with limit result; range date; without source data
qry = 'sendiri'
nResults = 10 # number of result
withresultdata = False # get result data with/without source data
dateStart, dateEnd = "16/07/2018", "26/07/2018" # "format": "dd/MM/yyyy||yyyy"
dateField = 'created_at'
Q = {"bool":{"must":{'multi_match':{'query':qry,"type":"cross_fields",'fields':['full_text']}},"filter": {"range":{dateField:{"gte":dateStart,"lte":dateEnd,"format":"dd/MM/yyyy||yyyy"}}}}}
# hsl = conEs.search(index=index_name, body={"query": {"match": {'full_text':qry}}})
hsl = conEs.search(index=index_name, size = nResults, _source= withresultdata ,doc_type=docType,body={"query":Q})
hsl['hits']['hits']
# hsl

[{u'_id': u'AWTU-A3z4W0kSSaFYMMs',
  u'_index': u'twitter',
  u'_score': 6.228952,
  u'_type': u'tweets'},
 {u'_id': u'AWTU-F6U4W0kSSaFYNBl',
  u'_index': u'twitter',
  u'_score': 6.0585947,
  u'_type': u'tweets'},
 {u'_id': u'AWTU975S4W0kSSaFYLYt',
  u'_index': u'twitter',
  u'_score': 5.983316,
  u'_type': u'tweets'},
 {u'_id': u'AWTU9_SO4W0kSSaFYL8C',
  u'_index': u'twitter',
  u'_score': 5.983316,
  u'_type': u'tweets'},
 {u'_id': u'AWTU-Hy64W0kSSaFYNVg',
  u'_index': u'twitter',
  u'_score': 5.983316,
  u'_type': u'tweets'},
 {u'_id': u'AWTU99J44W0kSSaFYLl_',
  u'_index': u'twitter',
  u'_score': 5.872378,
  u'_type': u'tweets'},
 {u'_id': u'AWTU9_b24W0kSSaFYL9p',
  u'_index': u'twitter',
  u'_score': 5.872378,
  u'_type': u'tweets'},
 {u'_id': u'AWTU-HZC4W0kSSaFYNRN',
  u'_index': u'twitter',
  u'_score': 5.4440975,
  u'_type': u'tweets'},
 {u'_id': u'AWTU-HM04W0kSSaFYNPK',
  u'_index': u'twitter',
  u'_score': 5.2313385,
  u'_type': u'tweets'},
 {u'_id': u'AWTU99XB4W0kSSaFYLoJ',

In [133]:
# "Term Vector" information?
# Get "_id" for query 
dID = hsl['hits']['hits'][9]['_id']
print(dID)

AWTU99XB4W0kSSaFYLoJ


In [138]:
# Get Term Vectors
termVec=conEs.termvectors(index=idx,doc_type=docType,id=dID,fields = ['full_text'],term_statistics=True)
termVec
# what is Term Vectors?
# https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-termvectors.html
# Kita juga bisa menggunakan "term position"
# Btw, good News TermVectors supported di ElasticSearch dengan license Full opensource Apache 2 :)
# "Statistics" dari term vectors ini secara realtime diupdate dan dapat digunakan untuk menghitung
# VSM/term Weight di TextMining seperti yang dibahas kemarin

{u'_id': u'AWTU99XB4W0kSSaFYLoJ',
 u'_index': u'twitter',
 u'_type': u'tweets',
 u'_version': 1,
 u'found': True,
 u'term_vectors': {u'full_text': {u'field_statistics': {u'doc_count': 4995,
    u'sum_doc_freq': 83224,
    u'sum_ttf': 90711},
   u'terms': {u'aku': {u'doc_freq': 4805, u'term_freq': 1, u'ttf': 6620},
    u'astrilystd': {u'doc_freq': 1, u'term_freq': 1, u'ttf': 1},
    u'depan': {u'doc_freq': 45, u'term_freq': 1, u'ttf': 46},
    u'https': {u'doc_freq': 772, u'term_freq': 1, u'ttf': 775},
    u'nwv6991phy': {u'doc_freq': 1, u'term_freq': 1, u'ttf': 1},
    u'sendiri': {u'doc_freq': 105, u'term_freq': 1, u'ttf': 110},
    u't.co': {u'doc_freq': 748, u'term_freq': 1, u'ttf': 751}}}},
 u'took': 0}